In [ ]:
import numpy as np
import time

# Function Definitions
def fx(x, lambda_reg):
    return 0.5 * np.linalg.norm(A @ x - y) ** 2 + 0.5 * lambda_reg * np.dot(x, x)

def grad_fx(x, lambda_reg):
    return np.dot(A.transpose(), A) @ x - A.transpose() @ y + lambda_reg * x

def hessian_fx(lambda_reg):
    return A.transpose() @ A + lambda_reg * np.identity(A.shape[1])

def dk_fx(lambda_reg):
    return np.linalg.inv(hessian_fx(lambda_reg))

def get_alpha_newton(xk, alpha0, rho, gamma, Dk, lambda_reg):
    alpha = alpha0
    pk = -grad_fx(xk, lambda_reg)
    while fx(xk + alpha * Dk @ pk, lambda_reg) > (fx(xk, lambda_reg) + gamma * alpha * grad_fx(xk, lambda_reg) @ Dk @ pk):
        alpha = rho * alpha
    return alpha

def newton_method_with_backtracking(x0, tau, alpha0, rho, gamma, lambda_reg):
    xk = np.copy(x0)
    count = 0
    pk = grad_fx(xk, lambda_reg)
    xks = [xk]
    while np.linalg.norm(pk) > tau:
        Dk = dk_fx(lambda_reg)
        alpha = get_alpha_newton(xk, alpha0, rho, gamma, Dk, lambda_reg)
        xk = xk - alpha * Dk @ pk
        pk = grad_fx(xk, lambda_reg)
        xks.append(xk)
        count += 1
    return count, xk, fx(xk, lambda_reg), xks

# Experiment Parameters
np.random.seed(10)  # for repeatability
N = 200
ds = [1000, 5000, 10000, 20000, 25000, 50000, 100000, 200000, 500000, 1000000]
lambda_reg = 0.001
eps = np.random.randn(N, 1)  # random noise

# Experiment Execution
for i in range(np.size(ds)):
    d = ds[i]
    A = np.random.randn(N, d)
    for j in range(A.shape[1]):
        A[:, j] = A[:, j] / np.linalg.norm(A[:, j])

    xorig = np.ones((d, 1)).flatten()
    x0 = np.zeros(d)
    y = np.dot(A, xorig) + eps.flatten()

    tau = 1e-5
    alpha0 = 0.99
    rho = 0.5
    gamma = 0.5

    start = time.time()
    count, minimizer, minimum, _ = newton_method_with_backtracking(x0, tau, alpha0, rho, gamma, lambda_reg)
    newtontime = time.time() - start

    print("For d:", d)
    print("Number of iterations:", count)
    print("Minimum value:", minimum, "using Newton method with backtracking")
    print("|| Ax_opt - y ||^2 =", np.linalg.norm(A @ xorig - y) ** 2)
    print("|| x_opt - xorig ||^2 =", np.linalg.norm(minimizer - xorig) ** 2)
    print("Time taken for Newton method:", newtontime)


For d: 1000
Number of iterations: 4
Minimum value: 0.14183476870809816 using Newton method with backtracking
|| Ax_opt - y ||^2 = 191.7145432980871
|| x_opt - xorig ||^2 = 826.8244356809709
Time taken for Newton method: 0.8585500717163086
For d: 5000
Number of iterations: 4
Minimum value: 0.10576241777383968 using Newton method with backtracking
|| Ax_opt - y ||^2 = 191.7145432980871
|| x_opt - xorig ||^2 = 4806.013431133197
Time taken for Newton method: 55.547008752822876
For d: 10000
Number of iterations: 4
Minimum value: 0.11443640710309748 using Newton method with backtracking
|| Ax_opt - y ||^2 = 191.71454329808705
|| x_opt - xorig ||^2 = 9780.120415921312
Time taken for Newton method: 392.1974811553955


In [ ]:
import numpy as np
import time

# Function Definitions
def objective_func(x, lambda_val):
    return 0.5 * np.linalg.norm(matrix_A @ x - vector_y) ** 2 + 0.5 * lambda_val * np.dot(x, x)

def gradient_func(x, lambda_val):
    return np.dot(matrix_A.transpose(), matrix_A) @ x - matrix_A.transpose() @ vector_y + lambda_val * x

def hessian_mat(lambda_val):
    return matrix_A.transpose() @ matrix_A + lambda_val * np.identity(matrix_A.shape[1])

def inv_hessian_mat(lambda_val):
    return np.linalg.inv(hessian_mat(lambda_val))

def step_size_bfgs(xk, alpha_start, rho_val, gamma_val, B_k, lambda_val):
    alpha = alpha_start
    pk = -gradient_func(xk, lambda_val)
    while objective_func(xk + alpha * B_k @ pk, lambda_val) > (objective_func(xk, lambda_val) + gamma_val * alpha * gradient_func(xk, lambda_val) @ B_k @ pk):
        alpha = rho_val * alpha
    return alpha

def bfgs_method(x_start, tol_val, alpha_start, rho_val, gamma_val, lambda_val):
    x_k = np.copy(x_start)
    n = len(x_start)
    B_k = np.eye(n)
    iteration_count = 0
    pk = gradient_func(x_k, lambda_val)
    x_k_list = [x_k]

    while np.linalg.norm(pk) > tol_val:
        alpha = step_size_bfgs(x_k, alpha_start, rho_val, gamma_val, B_k, lambda_val)
        x_next = x_k - alpha * (B_k @ pk)
        sk = x_next - x_k
        yk = gradient_func(x_next, lambda_val) - gradient_func(x_k, lambda_val)

        B_k = np.dot((np.eye(len(x_k)) - np.outer(sk, yk) / np.dot(yk, sk)), np.dot(B_k, (np.eye(len(x_k)) - np.outer(yk, sk) / np.dot(yk, sk)))) + np.outer(sk, sk) / np.dot(yk, sk)

        x_k = x_next
        pk = gradient_func(x_k, lambda_val)
        x_k_list.append(x_k)
        iteration_count += 1

    return iteration_count, x_k, objective_func(x_k, lambda_val), x_k_list

# Experiment Parameters
np.random.seed(10)
num_samples = 200
dimensions_list = [1000, 5000, 10000, 20000, 25000, 50000, 100000, 200000, 500000, 1000000]
lambda_val = 0.001
noise_eps = np.random.randn(num_samples, 1)

# Experiment Execution
for dim_val in dimensions_list:
    dim_size = dim_val
    matrix_A = np.random.randn(num_samples, dim_size)
    for j in range(matrix_A.shape[1]):
        matrix_A[:, j] = matrix_A[:, j] / np.linalg.norm(matrix_A[:, j])

    x_original = np.ones((dim_size, 1)).flatten()
    x_start = np.zeros(dim_size)
    vector_y = np.dot(matrix_A, x_original) + noise_eps.flatten()

    tolerance_val = 1e-5
    alpha_start_val = 0.99
    rho_val = 0.5
    gamma_val = 0.5

    start_time = time.time()

    iter_count, minimizer, min_val, _ = bfgs_method(x_start, tolerance_val, alpha_start_val, rho_val, gamma_val, lambda_val)
    exec_time = time.time() - start_time

    print("For dimension:", dim_size)
    print("Number of iterations:", iter_count)
    print("Minimum value:", min_val, "using BFGS method")
    print("||Ax_opt - y||^2 =", np.linalg.norm(matrix_A @ x_original - vector_y) ** 2)
    print("||x_opt - x_original||^2 =", np.linalg.norm(minimizer - x_original) ** 2)
    print("Time taken for BFGS method:", exec_time)


For dimension: 1000
Number of iterations: 32
Minimum value: 0.14183476871428477 using BFGS method
||Ax_opt - y||^2 = 191.7145432980871
||x_opt - x_original||^2 = 826.824435809269
Time taken for BFGS method: 9.607851505279541
For dimension: 5000
Number of iterations: 40
Minimum value: 0.10576241777506894 using BFGS method
||Ax_opt - y||^2 = 191.7145432980871
||x_opt - x_original||^2 = 4806.0134310008425
Time taken for BFGS method: 872.1767685413361
